In [28]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.utilities.data_transformation import load_all_files

in_dir = AbsDirPath.MERGED
df = load_all_files(in_dir)

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.0.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.1.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue.2.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\merged\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\

In [29]:
df.shape

(1601472, 20)

In [30]:
df.head(3)

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar,s0_to_eliminate,s0_reasoning,s1_true_positive,s1_reasoning,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1,False,This content describes a function that process...,True,The content describes functionality related to...,False,Step 1: Initial Analysis\nThe content describe...,None,None
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,1,True,This is a technical instruction or algorithm d...,None,None,None,None,None,None
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",3,2,False,This is a brief instruction about handling dow...,True,The content describes a mechanism for handling...,False,The content describes a specific implementatio...,None,None


# Add columns denoting which stage passed

In [31]:
df["prefilter_passed"] = ~df['s0_to_eliminate'].isna()
df['s0_passed'] = df["prefilter_passed"] & (df['s0_to_eliminate'] == False)
df['s1_passed'] = df['s0_passed'] & (df['s1_true_positive'] == True)
df['s2_passed'] = df['s1_passed'] & (df['s2_related_to_arch'] == True)
df['s3_passed'] = df['s2_passed'] & (~df['s3_tactic'].isna()) & (df['s3_tactic'] != "None")

In [32]:
df.columns

Index(['id', 'keyword', 'keyword_raw', 'sentence', 'qa', 'source', 'url',
       'repo_id', 'matched_word', 'unique_matched_words',
       'nunique_matched_words', 'nsimilar', 's0_to_eliminate', 's0_reasoning',
       's1_true_positive', 's1_reasoning', 's2_related_to_arch',
       's2_reasoning', 's3_tactic', 's3_response', 'prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed'],
      dtype='object')

# Add column for tactic category

In [33]:
from cfg.tactics.tactic_to_category_map import tactic_to_category_map

df["s3_tactic_category"] = df.s3_tactic.apply(lambda x: tactic_to_category_map.get(x))
df.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,...,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response,prefilter_passed,s0_passed,s1_passed,s2_passed,s3_passed,s3_tactic_category
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,Step 1: Initial Analysis\nThe content describe...,None,None,True,True,True,False,False,None
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,None,None,None,None,True,False,False,False,False,None
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,The content describes a specific implementatio...,None,None,True,True,True,False,False,None
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",...,True,The content describes a candidate generator fo...,None,The system is implementing a candidate generat...,True,True,True,True,False,None
4,allenai/scispacy/v0.5.5.code_comment.availabil...,consist,consist,A spacy pipeline component which identifies en...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,consists,"consists, Consists",...,None,None,None,None,True,False,False,False,False,None


In [35]:
df.groupby("s3_tactic_category").nsimilar.agg("sum")

s3_tactic_category
Control Resource Demand                       1763
Control and Observe System State              1261
Defer Binding                                   59
Detect Attacks                                  33
Detect Faults                                  180
Increase Cohesion                               17
Limit Complexity                               202
Manage Interfaces                             1930
Manage Resources                              3189
Prevent Faults                                1260
React to Attacks                                20
Recover from Attacks                             2
Recover from Faults_Preparation and Repair      82
Recover from Faults_Reintroduction               1
Reduce Coupling                                206
Reduce Size of a Module                        207
Resist Attacks                                1072
Resource Adaptation                           3056
Resource Allocation                            179
Resource Mon

# R1
## Primary - A quantitative analysis of the prevalence of each quality attribute.

In [46]:
r1_total_count = df[df.s1_passed].nsimilar.sum()
r1_total_count

np.int64(620919)

In [56]:
r1 = df[df.s1_passed].groupby("qa").agg(nsimilar=("nsimilar","sum")).sort_values("nsimilar", ascending=False)
r1["percentage"] = r1.nsimilar / r1_total_count * 100
r1

,nsimilar,percentage
qa,,
usability,81624,13.145676
safety,74280,11.962913
deployability,69972,11.269103
integrability,55903,9.003268
performance,54407,8.762335
modifiability,53923,8.684386
testability,50777,8.177717
security,50093,8.067558
interoperability,43368,6.984486


In [62]:
import plotly.express as px
px.pie(r1, names=r1.index, values="nsimilar", title="R1")

## Secondary - Repository-Specific Priorities

In [77]:
r1_2 = df[df.s1_passed].pivot_table(index=["repo_id"], columns="qa", values="nsimilar", aggfunc="sum")
r1_2

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability
repo_id,,,,,,,,,,,,
OpenGene/fastp/v0.23.4,393,663,590,1046,742,754,467,142,790,440,269,1111
allenai/scispacy/v0.5.5,292,875,464,637,313,424,513,117,498,548,346,718
google/deepvariant/v1.6.1,1692,4075,2320,2286,1676,1696,2824,604,3353,2153,2468,5830
root-project/root/v6-32-06,27648,57117,33905,46686,37623,46281,46662,12110,63228,43588,42232,65132
scverse/scanpy/1.10.2,3018,7242,2241,5248,3014,4768,3941,1036,6411,3364,5462,8833


In [75]:
r1_2_pc = r1_2.div(r1_2.sum(axis=1), axis=0) * 100
r1_2_pc

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability
repo_id,,,,,,,,,,,,
OpenGene/fastp/v0.23.4,5.391186,9.317192,7.885953,15.155287,9.730158,10.007354,6.528257,2.517395,10.759744,6.098320,3.252814,13.356339
allenai/scispacy/v0.5.5,5.345201,17.373081,7.031176,8.279175,5.491193,8.250918,7.860036,1.770745,10.869360,9.183385,5.340492,13.205237
google/deepvariant/v1.6.1,6.056916,12.697135,6.376063,8.255592,6.323036,5.741697,9.854273,2.489345,13.275527,6.906338,7.210755,14.813324
root-project/root/v6-32-06,5.393953,12.130909,7.184916,8.940985,7.423804,8.325616,8.321707,2.234079,12.140654,8.914785,7.218292,11.770301
scverse/scanpy/1.10.2,5.715650,16.824861,4.290478,8.808272,5.561731,8.818960,6.823721,1.864837,12.111819,6.083344,8.880955,14.215372


In [76]:
px.bar(r1_2_pc, title="R1_2")

## Secondary 2 - Keyword Performance Analysis

In [80]:
df.head()

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,...,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response,prefilter_passed,s0_passed,s1_passed,s2_passed,s3_passed,s3_tactic_category
0,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,Given something that might be a URL (or might ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,Step 1: Initial Analysis\nThe content describe...,None,None,True,True,True,False,False,None
1,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Given a URL, look for the corresponding datase...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,None,None,None,None,True,False,False,False,False,None
2,allenai/scispacy/v0.5.5.code_comment.availabil...,down,down,"Download to temporary file, then copy to cache...",availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,download,"download, Download, DOWNLOAD",...,False,The content describes a specific implementatio...,None,None,True,True,True,False,False,None
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",...,True,The content describes a candidate generator fo...,None,The system is implementing a candidate generat...,True,True,True,True,False,None
4,allenai/scispacy/v0.5.5.code_comment.availabil...,consist,consist,A spacy pipeline component which identifies en...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,consists,"consists, Consists",...,None,None,None,None,True,False,False,False,False,None


In [92]:
r1_3 = df.pivot_table(index="keyword", columns="s1_passed", values="nsimilar", aggfunc="sum")
r1_3["total"] = r1_3.sum(axis=1)
nkw =  r1_3.total.sum()
r1_3["passed_ratio"] = r1_3[True] / r1_3.total
r1_3["popularity_ratio"] = r1_3.total / nkw
r1_3.sort_values("total", ascending=False)

s1_passed,False,True,total,passed_ratio,popularity_ratio
keyword,,,,,
error,91251.0,35728.0,126979.0,0.281369,6.365341e-02
test,54812.0,26291.0,81103.0,0.324168,4.065619e-02
depend,41351.0,23573.0,64924.0,0.363086,3.254581e-02
version,43375.0,20494.0,63869.0,0.320876,3.201694e-02
modul,44973.0,17002.0,61975.0,0.274336,3.106750e-02
...,...,...,...,...,...
tenant,1.0,NaN,1.0,NaN,5.012908e-07
trojan,NaN,1.0,1.0,1.000000,5.012908e-07
guardrail,NaN,1.0,1.0,1.000000,5.012908e-07


In [139]:
px.scatter(r1_3, x="popularity_ratio", y="passed_ratio", hover_name=r1_3.index, hover_data=["passed_ratio", "popularity_ratio", "total"])
# px.scatter(r1_3, x="popularity_ratio", y="passed_ratio", hover_name=r1_3.index, hover_data=["passed_ratio", "popularity_ratio", "total"], color="total", color_continuous_scale=px.colors.sequential.Burgyl_r)

# RQ2
## Primary - A strategic analysis of the types of solutions (tactic categories) used for the most important quality attributes. This gives a high-level overview of the solution strategies

In [103]:
data_rq2 = df[df.s3_passed]
data_rq2.shape

(12077, 26)

In [111]:
top_qas = df[df.s1_passed].groupby("qa").nsimilar.sum().sort_values(ascending=False).head(4)
top_qas = top_qas.index.tolist()
top_qas

['usability', 'safety', 'deployability', 'integrability']

In [137]:
r2 = data_rq2[data_rq2.qa.isin(top_qas)]
r2.groupby(["qa", "s3_tactic_category"]).nsimilar.sum().reset_index().sort_values(["qa","nsimilar"], ascending=[False]*2).groupby("qa").head(3)

,qa,s3_tactic_category,nsimilar
17,usability,Resist Attacks,819
19,usability,Support System Initiative,157
20,usability,Support User Initiative,56
14,safety,Prevent Faults,1170
13,safety,Detect Faults,138
16,safety,Resist Attacks,61
9,integrability,Manage Interfaces,1056
6,integrability,Control Resource Demand,71
7,integrability,Control and Observe System State,14
4,deployability,Resource Adaptation,1342


## Secondary - A detailed "drill-down" to see which specific tactics make up the most important categories identified in the primary analysis.

### TODO

In [138]:
r2 = data_rq2[data_rq2.qa.isin(top_qas)]
r2.groupby(["qa", "s3_tactic"]).nsimilar.sum().reset_index().sort_values(["qa","nsimilar"], ascending=[False]*2).groupby("qa").head(3)

,qa,s3_tactic,nsimilar
45,usability,Separate Entities,819
41,usability,Maintain system models,120
42,usability,Maintain task models,35
33,safety,Removal from Service,1112
24,safety,Condition Monitoring,93
31,safety,Predictive Model,58
21,integrability,Tailor Interface,945
19,integrability,Orchestrate,111
18,integrability,Manage Sampling Rate,71
7,deployability,Reduce Overhead,1342


# RQ3
## Primary -  A comparative analysis of discussion topics across different communication channels (source).

In [140]:
data_rq3 = df[df['s3_passed']]
data_rq3.shape

(12077, 26)

In [152]:
r3 = data_rq3.pivot_table(index="qa", columns="source", values="nsimilar", aggfunc="sum", fill_value=0)
totals = r3.sum(axis=1)
r3 = r3.div(totals, axis=0) * 100
r3


source,code_comment,docs,issue,issue_comment,release,wiki
qa,,,,,,
availability,38.709677,25.000000,29.838710,6.451613,0.000000,0.000000
deployability,23.337438,12.192118,51.600985,12.253695,0.369458,0.246305
energy efficiency,29.931973,16.326531,43.990930,8.390023,1.133787,0.226757
integrability,32.460733,20.855148,35.863874,10.209424,0.087260,0.523560
interoperability,29.968783,22.580645,34.235172,12.486993,0.312175,0.416233
modifiability,23.071672,10.921502,56.109215,9.419795,0.341297,0.136519
performance,25.920900,21.306708,30.166731,21.675068,0.484684,0.445909
reliability,38.805970,22.388060,19.402985,19.402985,0.000000,0.000000
safety,21.785714,15.785714,50.357143,11.928571,0.000000,0.142857


In [153]:
r3_2 = data_rq3.pivot_table(index="s3_tactic_category", columns="source", values="nsimilar", aggfunc="sum", fill_value=0)
totals = r3_2.sum(axis=1)
r3_2 = r3_2.div(totals, axis=0) * 100
r3_2

source,code_comment,docs,issue,issue_comment,release,wiki
s3_tactic_category,,,,,,
Control Resource Demand,32.047646,22.178106,23.425978,21.837777,0.397050,0.113443
Control and Observe System State,27.676447,17.129262,34.258525,20.539255,0.237906,0.158604
Defer Binding,22.033898,33.898305,20.338983,23.728814,0.000000,0.000000
Detect Attacks,39.393939,3.030303,42.424242,15.151515,0.000000,0.000000
Detect Faults,47.777778,27.222222,13.333333,11.666667,0.000000,0.000000
Increase Cohesion,0.000000,17.647059,52.941176,29.411765,0.000000,0.000000
Limit Complexity,31.188119,9.405941,45.544554,13.861386,0.000000,0.000000
Manage Interfaces,31.709845,18.704663,37.409326,11.450777,0.207254,0.518135
Manage Resources,23.047977,23.769207,30.072123,21.919097,0.564440,0.627156


# Other

In [4]:
df.source.unique()

array(['code_comment', 'docs', 'issue', 'issue_comment', 'release',
       'wiki'], dtype=object)

In [5]:
df.qa.unique()

array(['availability', 'deployability', 'energy efficiency',
       'integrability', 'interoperability', 'modifiability',
       'performance', 'reliability', 'safety', 'security', 'testability',
       'usability'], dtype=object)

In [6]:
df.repo_id.unique()

array(['allenai/scispacy/v0.5.5', 'google/deepvariant/v1.6.1',
       'OpenGene/fastp/v0.23.4', 'root-project/root/v6-32-06',
       'scverse/scanpy/1.10.2'], dtype=object)

In [7]:
df.columns

Index(['id', 'keyword', 'keyword_raw', 'sentence', 'qa', 'source', 'url',
       'repo_id', 'matched_word', 'unique_matched_words',
       'nunique_matched_words', 'nsimilar', 's0_to_eliminate', 's0_reasoning',
       's1_true_positive', 's1_reasoning', 's2_related_to_arch',
       's2_reasoning', 's3_tactic', 's3_response', 'prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed'],
      dtype='object')

# Data funnel

In [13]:
funnel = df.groupby(['prefilter_passed',
       's0_passed', 's1_passed', 's2_passed', 's3_passed']).nsimilar.agg("sum")
funnel

prefilter_passed  s0_passed  s1_passed  s2_passed  s3_passed
False             False      False      False      False         103326
True              False      False      False      False        1220672
                  True       False      False      False          49933
                             True       False      False         525789
                                        True       False          80045
                                                   True           15085
Name: nsimilar, dtype: int64

In [149]:
import pandas as pd

# Define the order and names of your funnel stages
stages = {
    'prefilter_passed': '1. Prefilter',
    's0_passed': '2. Stage 0',
    's1_passed': '3. Stage 1',
    's2_passed': '4. Stage 2',
    's3_passed': '5. Stage 3'
}

# This list will hold the data for each stage
funnel_data_list = []

# Process each stage
for stage_col, stage_name in stages.items():
    # 1. Filter for items that passed the current stage
    passed_stage_df = df[df[stage_col] == True]

    # 2. Group by 'source' and sum 'nsimilar'
    stage_summary = passed_stage_df.groupby('source')['nsimilar'].sum().reset_index()

    # 3. Add the stage name for plotting
    stage_summary['Stage'] = stage_name

    funnel_data_list.append(stage_summary)

# 4. Combine all stage data into a single DataFrame
funnel_plot_df = pd.concat(funnel_data_list, ignore_index=True)

print("Reshaped data for plotting:")
print(funnel_plot_df)

Reshaped data for plotting:
           source  nsimilar         Stage
0    code_comment    968267  1. Prefilter
1            docs    260477  1. Prefilter
2           issue    308571  1. Prefilter
3   issue_comment    331911  1. Prefilter
4         release      1099  1. Prefilter
5            wiki     21199  1. Prefilter
6    code_comment    306518    2. Stage 0
7            docs     71142    2. Stage 0
8           issue    130079    2. Stage 0
9   issue_comment    159928    2. Stage 0
10        release       699    2. Stage 0
11           wiki      2486    2. Stage 0
12   code_comment    288182    3. Stage 1
13           docs     68007    3. Stage 1
14          issue    122615    3. Stage 1
15  issue_comment    139048    3. Stage 1
16        release       641    3. Stage 1
17           wiki      2426    3. Stage 1
18   code_comment     18638    4. Stage 2
19           docs     17077    4. Stage 2
20          issue     28888    4. Stage 2
21  issue_comment     29756    4. Stage 2
22    

In [150]:
px.funnel(funnel_plot_df,
                x='nsimilar',
                y='Stage',
                color='source',
                title='Funnel Analysis by Source')

In [14]:
total_items = df.nsimilar.sum()
funnel_df = funnel.reset_index()
funnel_df["percentage_of_total"] = funnel_df.nsimilar / total_items * 100
funnel_df

,prefilter_passed,s0_passed,s1_passed,s2_passed,s3_passed,nsimilar,percentage_of_total
0,False,False,False,False,False,103326,5.179638
1,True,False,False,False,False,1220672,61.191167
2,True,True,False,False,False,49933,2.503095
3,True,True,True,False,False,525789,26.357320
4,True,True,True,True,False,80045,4.012582
5,True,True,True,True,True,15085,0.756197


## Crosstab

In [19]:
import pandas as pd

df_tactics = df[df['s3_passed'] == True]
tactic_crosstab = pd.crosstab(df_tactics['qa'], df_tactics['s3_tactic'])
tactic_crosstab

s3_tactic,Abstract Common Services,Abstract Data Sources,Active Redundancy,Aggregate,Authenticate Actors,Authorize Actors,Bound Execution Times,Bound Queue Sizes,Cancel,Change Default Settings,...,Startup Time Binding,Static Classification,Tailor Interface,Timestamp,Transactions,Undo,Use an Intermediary,Verify Message Integrity,Vertical Scaling,Voting
qa,,,,,,,,,,,,,,,,,,,,,
availability,0,0,5,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0
deployability,10,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,5,0,0,0
energy efficiency,0,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,26,0
integrability,2,0,0,0,0,0,0,0,0,0,...,1,0,735,0,0,0,0,0,0,0
interoperability,1,0,0,0,0,0,0,0,0,0,...,1,0,635,0,0,0,0,0,0,0
modifiability,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,0,0,0
performance,0,0,0,0,0,0,1086,2252,0,0,...,0,0,0,0,0,0,0,0,0,0
reliability,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
safety,0,0,6,0,0,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [22]:
import pandas as pd

df_tactics = df[df['s1_passed']]
tactic_crosstab = pd.crosstab(df_tactics['source'], df_tactics['qa'], margins=True)
tactic_crosstab

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability,All
source,,,,,,,,,,,,,
code_comment,11808,18901,16894,21111,18108,20759,20169,5130,26947,21739,16788,27694,226048
docs,3167,6925,3802,5853,5717,5726,5676,1217,6101,5148,5009,9883,64224
issue,4738,12865,5184,7626,4972,7428,7036,2248,12372,6798,8507,13006,92780
issue_comment,6162,16288,5658,9728,6295,8469,9450,3313,14574,7822,11090,16540,115389
release,22,114,34,33,25,36,45,11,49,58,26,76,529
wiki,135,301,126,323,191,178,146,34,159,206,169,382,2350
All,26032,55394,31698,44674,35308,42596,42522,11953,60202,41771,41589,67581,501320


In [27]:
tactic_crosstab_totals = tactic_crosstab.loc["All"]
tactic_crosstab.div(tactic_crosstab_totals, axis=1) * 100

qa,availability,deployability,energy efficiency,integrability,interoperability,modifiability,performance,reliability,safety,security,testability,usability,All
source,,,,,,,,,,,,,
code_comment,45.359557,34.121024,53.296738,47.255674,51.285828,48.734623,47.431918,42.918096,44.760971,52.043284,40.366443,40.978973,45.090561
docs,12.165796,12.501354,11.994448,13.101580,16.191798,13.442577,13.348384,10.181544,10.134215,12.324340,12.044050,14.623933,12.810979
issue,18.200676,23.224537,16.354344,17.070332,14.081794,17.438257,16.546729,18.806994,20.550812,16.274449,20.454928,19.245054,18.507141
issue_comment,23.670867,29.403907,17.849707,21.775529,17.828821,19.882149,22.223790,27.716891,24.208498,18.725910,26.665705,24.474335,23.017035
release,0.084511,0.205798,0.107262,0.073868,0.070805,0.084515,0.105828,0.092027,0.081393,0.138852,0.062517,0.112458,0.105521
wiki,0.518593,0.543380,0.397501,0.723016,0.540954,0.417880,0.343352,0.284447,0.264111,0.493165,0.406357,0.565248,0.468762
All,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [13]:
df.pivot_table(index=["source", "qa"], columns=["s0_to_eliminate", "s1_true_positive", "s2_related_to_arch"], values="nsimilar", aggfunc="sum")

s0_to_eliminate                 False      
s1_true_positive                 True      
s2_related_to_arch              False True 
source       qa                            
code_comment availability       14863   798
             deployability      22537  2058
             energy efficiency  20247  1429
             integrability      25333  1935
             interoperability   21498  1535
...                               ...   ...
wiki         reliability           24    11
             safety               103    57
             security             179    44
             testability          122    54
             usability            279   110

[72 rows x 2 columns]

In [14]:
df.pivot_table(index=["source", "qa"], columns=["s0_to_eliminate"], values="nsimilar", aggfunc=["sum"], margins=True)

sum                  
s0_to_eliminate                  False     True      All
source       qa                                         
code_comment availability        17010    37627    54637
             deployability       29456    65810    95266
             energy efficiency   22643    47049    69692
             integrability       28180    68841    97021
             interoperability    24650    53301    77951
...                                ...      ...      ...
wiki         safety                167     1283     1450
             security              238     1573     1811
             testability           176     1184     1360
             usability             390     2983     3373
All                             670852  1220672  1891524

[73 rows x 3 columns]

In [19]:
df.groupby(["s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
False            True              False               NaN                          525789
                                   True                Abstract Common Services         64
                                                       Abstract Data Sources          1153
                                                       Active Redundancy                13
                                                       Aggregate                        26
                                                                                    ...   
                                                       Vertical Scaling                 31
                                                       Voting                            3
                 False             NaN                 NaN                           49933
True             NaN               NaN                 NaN                         1220672
NaN       

In [15]:
df.groupby(["source", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

source        s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
code_comment  False            True              False               NaN                         269544
                                                 True                Abstract Common Services        38
                                                                     Abstract Data Sources          307
                                                                     Active Redundancy                3
                                                                     Aggregate                        2
                                                                                                  ...  
wiki          False            True              True                Removal from Service             1
                                                                     Separate Entities                4
                                                                     Tailo

In [16]:
df.groupby(["source", "qa", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"], dropna=False).nsimilar.agg("sum")

source        qa            s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic             
code_comment  availability  False            True              False               NaN                       14863
                                                               True                Active Redundancy             3
                                                                                   Condition Monitoring          6
                                                                                   Degradation                   2
                                                                                   Exception Detection           1
                                                                                                             ...  
wiki          usability     False            True              True                Maintain system models        2
                                                                                   None  

In [17]:
df.groupby(["source", "s0_to_eliminate", "s1_true_positive", "s2_related_to_arch", "s3_tactic"]).nsimilar.agg("sum")

source        s0_to_eliminate  s1_true_positive  s2_related_to_arch  s3_tactic               
code_comment  False            True              True                Abstract Common Services     38
                                                                     Abstract Data Sources       307
                                                                     Active Redundancy             3
                                                                     Aggregate                     2
                                                                     Authenticate Actors          11
                                                                                                ... 
wiki          False            True              True                Predictive Model              1
                                                                     Reduce Overhead               7
                                                                     Removal from Service         

In [18]:
df[df["s3_tactic"].notna()]

,id,keyword,keyword_raw,sentence,qa,source,url,repo_id,matched_word,unique_matched_words,nunique_matched_words,nsimilar,s0_to_eliminate,s0_reasoning,s1_true_positive,s1_reasoning,s2_related_to_arch,s2_reasoning,s3_tactic,s3_response
3,allenai/scispacy/v0.5.5.code_comment.availabil...,avail,avail,A candidate generator for entity linking to a ...,availability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,available,"available, Available, AVAILABLE",3,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is implementing a candidate generat...
8,allenai/scispacy/v0.5.5.code_comment.deployabi...,contain,contain,in a pre-indexed approximate nearest neighbou...,deployability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,contain,"contain, Contain",2,1,False,This content describes a technical system and ...,True,The content describes how an approximate neare...,True,The content describes a system-level component...,None,The system describes an approximate nearest ne...
53,allenai/scispacy/v0.5.5.code_comment.energy ef...,current,current,A candidate generator for entity linking to a ...,energy efficiency,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,currently,"Currently, currently, CURRENTLY",3,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
85,allenai/scispacy/v0.5.5.code_comment.integrabi...,configur,configur,A candidate generator for entity linking to a ...,integrability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,configured,"configured, Configured",2,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
86,allenai/scispacy/v0.5.5.code_comment.integrabi...,sub,sub,A candidate generator for entity linking to a ...,integrability,code_comment,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,subject,"Subject, subject",2,1,False,This content describes a technical system for ...,True,The content describes how the candidate genera...,True,The content describes a candidate generator fo...,None,The system is being described as a candidate g...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,scverse/scanpy/1.10.2.wiki.usability.c6c00c0ca...,help,help,API. API. Discussion . Need help? Reach out o...,usability,wiki,https://scanpy.readthedocs.io/en/stable/index....,scverse/scanpy/1.10.2,helping,helping,1,1,False,"This content is a mix of API documentation, re...",True,The content describes how users can get help t...,True,Step 1: The core topic of the content is about...,None,The system is providing information about the ...
2426,scverse/scanpy/1.10.2.wiki.usability.c9ae0abe1...,guid,guid,1.5.0 2020-05-15#. The 1.5.0 release adds a lo...,usability,wiki,https://scanpy.readthedocs.io/en/stable/releas...,scverse/scanpy/1.10.2,guide,"guide, Guide",2,1,False,This content describes a software release (ver...,True,The content describes new functionality added ...,True,The content describes updates and new function...,None,The system is being described in terms of its ...
2471,scverse/scanpy/1.10.2.wiki.usability.d81ef3b6a...,document,document,n on this while working on a PR:. Only run a s...,usability,wiki,https://scanpy.readthedocs.io/en/stable/dev/te...,scverse/scanpy/1.10.2,documentation,"documentation, Documentation, DOCUMENTATION",3,1,False,This content appears to be instructions and ad...,True,The content describes how to run tests with sp...,True,**Step 1